ABANDONED
=========

Hugging Face Trainer helper much more usable
--------------------------------------------

In [1]:
import pandas as pd
import mapply
mapply.init(progressbar=True, max_chunks_per_worker=100)

/home/hlt/HLT_PROJECT/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = "distilbert/distilroberta-base"

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case = True)

MODEL_DIR = '../NAS/custom__' + MODEL.split('/')[-1]
import os
os.makedirs(MODEL_DIR, exist_ok=True)

In [3]:
import os
if 'tokens.feather' not in os.listdir(MODEL_DIR):
  clean = pd.read_feather('../data/lyrics_cleaned.feather')
  clean = clean[clean['tag'] != 'misc']
  clean['tokens'] = clean['lyrics_clean'].mapply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=1000, truncation=True))

  clean.to_feather(MODEL_DIR + '/tokens.feather')




---




In [4]:
lyrics = pd.read_feather(MODEL_DIR + '/tokens.feather')
lyrics = lyrics.sample(frac=.5)

# sample_list = []
# for cls in lyrics['tag'].unique():
#   l = lyrics[lyrics['tag'] == cls]
#   sample_list += [l.sample(n=150)]
# lyrics = pd.concat(sample_list)

ONE_CLASS = ''

if ONE_CLASS != '':
  positive = lyrics[lyrics['tag'] == ONE_CLASS]
  negative = lyrics[lyrics['tag'] != ONE_CLASS].sample(n=positive.shape[0])
  negative['tag'] = f'not_{ONE_CLASS}'
  lyrics = pd.concat([positive, negative])

In [5]:
from torch.nn.utils.rnn import pad_sequence
import torch

PADDING = tokenizer.pad_token_id # roberta
# PADDING = 0 # llama


X_tensor_full = pad_sequence(
  [torch.tensor(x) for x in lyrics['tokens']],
  batch_first=True,
  padding_value=PADDING
)
attention_full = X_tensor_full != PADDING
attention_full = attention_full.to(torch.float32)

X_tensor = X_tensor_full[:, :512]
attention = attention_full[:, :512]

y_tensor = torch.tensor(lyrics['tag'].astype('category').cat.codes.tolist())

In [6]:
TRAIN_SIZE = 0.6
VAL_SIZE = 0.2
TEST_SIZE = 0.2

tot_len = len(X_tensor)
torch.random.manual_seed(42)
shuffle = torch.randperm(tot_len)

X_tensor = X_tensor[shuffle]
attention = attention[shuffle]
y_tensor = y_tensor[shuffle]

train_size = int(len(X_tensor) * TRAIN_SIZE)
val_size = int(len(X_tensor) * VAL_SIZE)
test_size = len(X_tensor) - train_size - val_size

X_train = X_tensor[:train_size]
X_val = X_tensor[train_size:train_size + val_size]
X_test = X_tensor[train_size + val_size:]

attention_train = attention[:train_size]
attention_val = attention[train_size:train_size + val_size]
attention_test = attention[train_size + val_size:]

y_train = y_tensor[:train_size]
y_val = y_tensor[train_size:train_size + val_size]
y_test = y_tensor[train_size + val_size:]

In [7]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 16

train_data = TensorDataset(X_train, attention_train, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(X_val, attention_val, y_val)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [8]:
from transformers import AutoModelForSequenceClassification, AdamW, AutoModelWithLMHead, AutoModel

model = AutoModelForSequenceClassification.from_pretrained(
  MODEL,
  num_labels=lyrics['tag'].nunique(),
  output_attentions=False,
  output_hidden_states=False,
)

model.cuda();

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
optimizer = AdamW(model.parameters(), lr=5e-4)

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

/home/hlt/HLT_PROJECT/.env/lib/python3.11/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
for name, param in model.named_parameters():
  if 'classifier' not in name:
    param.requires_grad = True
  else:
    param.requires_grad = True
    print(name, param.requires_grad)

classifier.dense.weight True
classifier.dense.bias True
classifier.out_proj.weight True
classifier.out_proj.bias True


In [11]:
a = None
for name, param in model.named_parameters():
  if 'classifier' not in name and 'pooler' not in name:
    # continue
    pass
  # print(name)
  # print(param.shape)

In [12]:
import random
import numpy as np
from tqdm import tqdm
from torcheval.metrics.functional import multiclass_f1_score


device = torch.device("cuda")

import datetime
CKP_DIR = MODEL_DIR + f"/{datetime.datetime.now()}"
os.makedirs(CKP_DIR, exist_ok=True)

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    nb_train_steps = 0
    train_f1 = torch.zeros(lyrics['tag'].nunique())

    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()


    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss = 0
    eval_f1 = torch.zeros(lyrics['tag'].nunique())
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in tqdm(val_dataloader, total=len(val_dataloader)):
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/160bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            attention_mask=b_input_mask)
        
        pred = outputs.logits.detach().cpu().argmax(axis=1)
        true = b_labels.to('cpu')

        tmp_f1 = multiclass_f1_score(true, pred, num_classes=lyrics['tag'].nunique(), average=None)
        
        # Accumulate the total accuracy.
        eval_f1 += tmp_f1

        # Track the number of batches
        nb_eval_steps += 1
    torch.save({
            'epoch': epoch_i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_train_loss,
    },
    CKP_DIR + f"/checkpoint_{epoch_i}.pt")    

    # Report the final accuracy for this validation run.
    f1 = eval_f1/nb_eval_steps
    for i in range(lyrics['tag'].nunique()):
      print(f'  F1 for {lyrics["tag"].astype("category").cat.categories[i]}: {f1[i]:.2f}', end=' ')
    print(f'  Mean F1: {f1.mean():.2f}')
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...


100%|██████████| 1316/1316 [14:11<00:00,  1.55it/s]



  Average training loss: 1.62

Running Validation...


100%|██████████| 439/439 [02:19<00:00,  3.16it/s]


  F1 for country: 0.00   F1 for pop: 0.00   F1 for rap: 0.00   F1 for rb: 0.33   F1 for rock: 0.00   Mean F1: 0.07

======== Epoch 2 / 3 ========
Training...


 74%|███████▍  | 980/1316 [10:36<03:38,  1.54it/s]